# SHA0 / SHA1
** Implementation of the RFC-3174 US Secure Hash Algorithm 0 and 1 **

*... in Python*

[SHA1](https://www.ietf.org/rfc/rfc3174.txt)

### Define and Select Test Cases

In [1]:
COMPUTE_SHA0 = False # Set to False to compute SHA1
algo_name = "SHA-0" if (COMPUTE_SHA0) else "SHA-1"

In [2]:
# [PlainText, SHA0Digest, SHA1Digest]
test_case=[["","da39a3ee5e6b4b0d3255bfef95601890afd80709","???"],\
           ["a","34aa973cd4c4daa4f61eeb2bdbad27316534016f","???"],\
           ["abc","a9993e364706816aba3e25717850c26c9cd0d89d","164b8a914cd2a5e74c4f7ff082c4d97f1edf880"],\
           ["abcdbcdecdefdefgefghfghighijhijkijkljklmklmnlmnomnopnopq","84983e441c3bd26ebaae4aa1f95129e5e54670f1","d2516ee1acfa5baf33dfc1c471e438449ef134c8"],\
           ["abcdefghbcdefghicdefghijdefghijkefghijklfghijklmghijklmnhijklmnoijklmnopjklmnopqklmnopqrlmnopqrsmnopqrstnopqrstu",\
            "a49b2446a02c645bf419f995b67091253a04a259","???"]]
use_test_case = 3

# Assign test case to variables.
message = test_case[use_test_case][0]
ref_idx = 2 if(COMPUTE_SHA0) else 1
ref_hash = test_case[use_test_case][ref_idx]

### Step 1 - Append Padding Bits

The message to be hashed is padded to have a length equal to 8 bytes {64 bits} less than being a multiple of 64 bytes {512 bits}. The padding step is performed even if the message length is already of desired length. The padding bit string used is `1` followed by `0` - `100...000`

The padded message lengths is eventually 56 bytes {448 bits}, 120 bytes {960 bits}, 184 bytes {1472 bits}, 248 bytes {1984 bits} and so on.

The padding method is similar to MD4.

In [3]:
message_len = len(message)
message_len_bits = message_len * 8
print("Message Length : " + str(message_len) + " bytes {" + str(message_len_bits) + " bits}")

Message Length : 56 bytes {448 bits}


In [4]:
# Encode string to bytes
message_b = message.encode('utf-8')

In [5]:
# Calculate padding length
padding_len = 56 - message_len % 64
padding_len = 64 if (padding_len == 0) else padding_len
print("Padding Length : " + str(padding_len) + " bytes {" + str(padding_len * 8) + " bits}")

Padding Length : 64 bytes {512 bits}


In [6]:
# Display Padded Message, length and calculation.
message_mod448 = message_b + b'\x80' + b'\x00' * (padding_len-1)
print("Padded Message :\n"+str(message_mod448))
print("\nlength(paddedMessage)      : "+str(len(message_mod448))+" bytes {"+str(len(message_mod448*8))+" bits}\nlength(paddedMessage) % 64 : "+str(len(message_mod448)%64)+" bytes {"+str((len(message_mod448)%64) * 8)+" bits}" )

Padded Message :
b'abcdbcdecdefdefgefghfghighijhijkijkljklmklmnlmnomnopnopq\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

length(paddedMessage)      : 120 bytes {960 bits}
length(paddedMessage) % 64 : 56 bytes {448 bits}


### Step 2 - Append Length

The bit length of the original message is appended to the padded message created in the previous step. This bit length is appended as an 8 byte {64 bits} little endian integer.

A message of length 14 bytes (_try test case # 3_), would have a bit length of 112 bits and the appended 64 bit little endian bit length would be `0x7000000000000000` (as hex) or `b'p\x00\x00\x00\x00\x00\x00\x00'` (as a byte string).

If the message length is $> 2^{64}$ bits, only the lower 64 bits are used for padding.

In [7]:
# Append Length
processed_message=message_mod448+(message_len_bits%2**64).to_bytes(8,byteorder='big')
print("LSB64(len(unPaddedMessage)) : "+str((message_len_bits%2**64).to_bytes(8,byteorder='big')))
print("length( paddedMessage | LSB64(len(unPaddedMessage)) ) : "+str(len(processed_message))+" bytes {"+str(len(processed_message)*8)+" bits}")
print("\nPadded Message | LSB64(len(unPaddedMessage)) :\n"+str(processed_message))

LSB64(len(unPaddedMessage)) : b'\x00\x00\x00\x00\x00\x00\x01\xc0'
length( paddedMessage | LSB64(len(unPaddedMessage)) ) : 128 bytes {1024 bits}

Padded Message | LSB64(len(unPaddedMessage)) :
b'abcdbcdecdefdefgefghfghighijhijkijkljklmklmnlmnomnopnopq\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\xc0'


### Step 3 - Initialize Context

These numbers are defined in the standards - RFC-3174, FIPS 180-1 for SHA1 and FIPS-180 for SHA0

In [8]:
h0 = 0x67452301
h1 = 0xEFCDAB89
h2 = 0x98BADCFE
h3 = 0x10325476
h4 = 0xC3D2E1F0

### Define various constants
These constants are also defined in the standards documents.

In [9]:
# Constant Table
K = [0x5A827999] * 20 + [0x6ED9EBA1] * 20 + [0x8F1BBCDC] * 20 + [0xCA62C1D6] * 20

# Shift Table
R1_s = [7, 12, 17, 22] * 4
R2_s = [5, 9, 14, 20] * 4
R3_s = [4, 11, 16, 23] * 4
R4_s = [6, 10, 15, 21] * 4

# K table (to use a sub-string of the message)
R1_k = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
R2_k = [1, 6, 11, 0, 5, 10, 15, 4, 9, 14, 3, 8, 13, 2, 7, 12]
R3_k = [5, 8, 11, 14, 1, 4, 7, 10, 13, 0, 3, 6, 9, 12, 15, 2]
R4_k = [0, 7, 14, 5, 12, 3, 10, 1, 8, 15, 6, 13, 4, 11, 2, 9]

In [10]:
# Auxiliary functions


# Function F that take in 3x 32bit words and an index integer and return 1x 32bit word.
# The operations differs across various rounds.
def F(t, B, C, D):
    calc = 0
    if (0 <= t & t <= 19):
        calc = (B & C) | (D & ~B)
    elif (20 <= t & t <= 39):
        calc = B ^ C ^ D
    elif (40 <= t & t <= 59):
        calc = (B & C) | (B & D) | (C & D)
    elif (60 <= t & t <= 79):
        calc = B ^ C ^ D
    else:
        raise ValueError('t is not in the range 0<=j<=79 !')
    return calc


# Rotate Left - Ensure output is 32bit (& 0xFFFFFFFF)
def rotl(x, s):
    return ((x << s) | x >> (32 - s)) & 0xFFFFFFFF

In [11]:
# Split message to a word list.
def words(M):
    word_list = [0] * 80
    for i in range(0, 16):
        word_list[i] = int.from_bytes(M[i * 4:i * 4 + 4], byteorder='big')
    return word_list

In [12]:
# Display SHA Context - To see what is happening...
def disp_context(h0, h1, h2, h3, h4):
    print("h0...h4 : {:08x} {:08x} {:08x} {:08x} {:08x}".format(h0, h1, h2, h3, h4))
    return None

### Step 4 - Process Message in 16-Word Blocks

In [13]:
# Loop though the various 512 bit blocks of a long message.
print("Initial Context:")
disp_context(h0, h1, h2, h3, h4)
print("")

# For each 512 bit (64 byte, 16 32bit words)
for i in range(0, len(processed_message), 64):
    M = processed_message[i:i + 64]
    # Split into 16x 32bit words. The list includes blanks (16-79) for scheduled messages.
    W = words(M)
    print("PROCESSING bytes " + str(i) + "..." + str(i + 64))
    print("Message chunk being processed :\n" + str(M))
    
    # Schedule Messages - Create message words 17..79 using word 0..16
    for t in range(16, 80):
        if COMPUTE_SHA0:
            W[t] = W[t - 3] ^ W[t - 8] ^ W[t - 14] ^ W[t - 16]
        else:
            W[t] = rotl(W[t - 3] ^ W[t - 8] ^ W[t - 14] ^ W[t - 16], 1)
            
    # Assign context to Round variable.
    [A, B, C, D, E] = [h0, h1, h2, h3, h4]

    # Process scheduled message. Word 0..79
    for t in range(0, 80):
        T = (rotl(A, 5) + F(t, B, C, D) + E + W[t] + K[t]) & 0xFFFFFFFF
        [E, D, C, B, A] = [D, C, rotl(B, 30), A, T]
    
    # Update SHA Context Buffer after processing the 512 bit message part.
    [h0, h1, h2, h3, h4] = [(h0 + A) & 0xFFFFFFFF, (h1 + B) & 0xFFFFFFFF, (h2 + C) & 0xFFFFFFFF, (h3 + D) & 0xFFFFFFFF, (h4 + E) & 0xFFFFFFFF]

    # Display Updated SHA Context Buffers after processing the 512 bit message part.
    disp_context(h0, h1, h2, h3, h4)
    print()

Initial Context:
h0...h4 : 67452301 efcdab89 98badcfe 10325476 c3d2e1f0

PROCESSING bytes 0...64
Message chunk being processed :
b'abcdbcdecdefdefgefghfghighijhijkijkljklmklmnlmnomnopnopq\x80\x00\x00\x00\x00\x00\x00\x00'
h0...h4 : f4286818 c37b27ae 0408f581 84677148 4a566572

PROCESSING bytes 64...128
Message chunk being processed :
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\xc0'
h0...h4 : 84983e44 1c3bd26e baae4aa1 f95129e5 e54670f1



In [14]:
# Compute output hash from the SHA context buffers.
# The final SHA hash is the SHA Context buffers concatinated as -
# h0 | h1 | h2 | h3 | h4
output_int = h0 << 128 | h1 << 96 | h2 << 64 | h3 << 32 | h4

# The SHAx hash starts with the lowest order byte of A ... highest order byte of D
print(algo_name + " hash")
print("OUTPUT      : " + hex(output_int))
print("REF. Hash   : 0x" + ref_hash)

SHA-1 hash
OUTPUT      : 0x84983e441c3bd26ebaae4aa1f95129e5e54670f1
REF. Hash   : 0x84983e441c3bd26ebaae4aa1f95129e5e54670f1


## Compare with Python's `hashlib`

In [15]:
import hashlib

In [16]:
H = hashlib.new('SHA1')
H.update(message_b)
sha1hash = H.hexdigest()
print("Hashlib SHA1 : 0x" + sha1hash)

Hashlib SHA1 : 0x84983e441c3bd26ebaae4aa1f95129e5e54670f1


### References

[SHA1](https://www.ietf.org/rfc/rfc3174.txt)
2. [Wikipedia](https://en.wikipedia.org/wiki/SHA1)
3. [Rosetta Code](https://rosettacode.org/wiki/SHA1/Implementation#Python)
4. [Merkle Damgård construction](https://en.wikipedia.org/wiki/Merkle%E2%80%93Damg%C3%A5rd_construction)

In [17]:
import numpy as np
from math import gcd 

In [18]:
b = np.array([7,11,19,39,79,157,313])
n = 900
r = 37
pT = [4,2,5,3,1,7,6]
t = r * b % n
a = np.zeros(b.shape)

for i in range(len(pT)) :
    a[i] = t[pT[i]-1]
    
print(a)

def bit7(x) :
    t = bin(ord(x))[2:]
    if len(t) == 8 :
        t = t[1:]
    while len(t) < 7 :
        t = '0' + t
    return t

def encrypt(x) :
    x = bit7(x)
    x = [int(i) for i in x]
    print(x)
    y = [a[i] * x[i] for i in range(len(x))]
    print(y)
    return sum(y)

[543. 407. 223. 703. 259. 781. 409.]


In [19]:
encrypt('b')

[1, 1, 0, 0, 0, 1, 0]
[543.0, 407.0, 0.0, 0.0, 0.0, 781.0, 0.0]


1731.0

In [23]:
def bruteInverse(a,m) :
    if gcd(a,m) != 1 :
        return -1 
    for i in range(m) :
        if (i*a)%m == 1 :
            return i

In [24]:
s = 1731
n = 900
r = 37
pT = [4,2,5,3,1,7,6]
pT_ = {}
for i in range(len(pT)) :
    pT_[pT[i]] = i + 1
r_ = bruteInverse(r,n)
s_ = s * r_ % n
print(pT_)
bRev = list(b)
bRev.reverse()

def decrypt(s_):
    x_ = np.zeros(b.shape)
    for i in range(len(bRev)) :
        if bRev[i] <= s_ :
            s_ -= bRev[i]
            x_[i] = 1
        else :
            x_[i] = 0
    temp = np.zeros(b.shape)
    print(x_)
    for i in range(len(pT)) :
        temp[i] = x_[pT_[i+1]-1]
    
    print(temp)
    return chr(int('0b'+''.join([str(int(i)) for i in temp]),2))

{4: 1, 2: 2, 5: 3, 3: 4, 1: 5, 7: 6, 6: 7}


In [25]:
decrypt(s_)

[1. 0. 0. 1. 0. 1. 0.]
[0. 0. 1. 1. 0. 0. 1.]


'\x19'

In [26]:
r_

73

In [27]:
bit7('b')

'1100010'

In [28]:
pT

[4, 2, 5, 3, 1, 7, 6]